# Steps and Tasks

## 1. Import and analyse the data set.

In [1]:
import pandas as pd # read data file, data processing
import numpy as np # linear algebra
import matplotlib.pyplot as plt # plotting graph for EDA , Metrics analysis
%matplotlib inline
import seaborn as sns # plotting graph for EDA , Metrics analysis

In [2]:
pip install langdetect;

In [3]:
#pip install googletrans==3.1.0a0

In [4]:
pip install googletrans==4.0.0-rc1

### Load the data 

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
data_dir = "/content/drive/MyDrive/AIML/projects/Capstone-NLP-Ticketing/"
data_file_name = 'input_data.xlsx'
data_file_path = data_dir+data_file_name
data_file_path

'/content/drive/MyDrive/AIML/projects/Capstone-NLP-Ticketing/input_data.xlsx'

In [7]:
df_data = pd.read_excel( data_file_path)

### Exploring the given Data files &  Understanding the structure of data

In [8]:
df_data.head(10)

,Short description,Description,Caller,Assignment group
0,login issue,-verified user details.(employee# & manager na...,spxjnwir pjlcoqds,GRP_0
1,outlook,_x000D_\n_x000D_\nreceived from: hmjdrvpb.komu...,hmjdrvpb komuaywn,GRP_0
2,cant log in to vpn,_x000D_\n_x000D_\nreceived from: eylqgodm.ybqk...,eylqgodm ybqkwiam,GRP_0
3,unable to access hr_tool page,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0
5,unable to log in to engineering tool and skype,unable to log in to engineering tool and skype,eflahbxn ltdgrvkz,GRP_0
6,event: critical:HostName_221.company.com the v...,event: critical:HostName_221.company.com the v...,jyoqwxhz clhxsoqy,GRP_1
7,ticket_no1550391- employment status - new non-...,ticket_no1550391- employment status - new non-...,eqzibjhw ymebpoih,GRP_0
8,unable to disable add ins on outlook,unable to disable add ins on outlook,mdbegvct dbvichlg,GRP_0
9,ticket update on inplant_874773,ticket update on inplant_874773,fumkcsji sarmtlhy,GRP_0


Short Description and Description are input features and Assignment is Class / Output

Dataset has IT related terms

Data contains whitespace characters , that needs to be removed 

In [9]:
df_data['orig_desc'] =  df_data['Description']
df_data['orig_short_desc'] = df_data['Short description']

In [10]:
# Replace white space characters with space
df_data['Description'] = df_data['Description'].replace(r'\n',' ', regex=True). replace(r'_x000D_',' ', regex=True)
df_data['Short description'] = df_data['Short description'].replace(r'\n',' ', regex=True). replace(r'_x000D_',' ', regex=True)

In [11]:
import unicodedata

# Using normalize function returns the conventional form for the Unicode string unistr. 
# The valid values for form are ‘NFC’, ‘NFKC’, ‘NFD’, and ‘NFKD’.

def remove_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFC', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)


In [12]:
df_data['Description'] = df_data['Description'].astype(str).apply(remove_accents)
df_data['Short description'] = df_data['Short description'].astype(str).apply(remove_accents)


### Detect and Translate Non English data to English

#### Detect Language

In [13]:
from langdetect import detect
def det(x):
    try:
        lang = detect(x)
    except:
        lang = 'Other'
    return lang

df_data['Lang'] = df_data['Description'].apply(det)

In [14]:
df_data['Lang'] = df_data['Lang'].apply(lambda x: 'en' if x == 'Other' else x)

In [15]:
df_data['Lang'].value_counts()

en    7080
de     400
af     272
it     138
fr     117
no      73
nl      68
sv      55
es      50
ca      48
da      30
pl      29
pt      22
ro      21
sl      17
cy      13
tl      12
et      10
sq       8
so       7
hr       7
fi       6
id       4
lv       3
lt       3
cs       2
sk       2
hu       1
vi       1
sw       1
Name: Lang, dtype: int64

**Dataset has multi lang data**

In [16]:
from googletrans import Translator

translator = Translator()

# detection
import time
def Translate_text_To_en(df, col_name , trans_col_name):
  # iterate through each row and select 
  for index, row in df.iterrows():
    src_lang = row['Lang']
    if src_lang != 'en':
      time.sleep(1)
      try:
        text_tr = row[col_name]
        trans_text = translator.translate(text_tr).text
        df.loc[index, trans_col_name]=trans_text
      except Exception as e:
        print(e)
        print("index is", index , "text is ",text_tr )
  print(df)
  return df


In [17]:
df_data['Translated_ShortDesc'] = df_data['Short description']

In [18]:
df_data[df_data['Lang'] == 'de']['Translated_ShortDesc']

133        install driver in printer hr14 in HostName_769
223                             probleme mit bluescreen .
251     reset the password for fygrwuna gomcekzi on e-...
265                     netweaver funktioniert nicht mehr
270         neues passwort fr accountname tgryhu hgygrtui
                              ...                        
8414      langsamer rechner \berprfung \niptbwdq csenjruz
8420                                  mobiltelefon defekt
8425      ich kann mein erp passwort nicht zurck setzten.
8439    der drucker fr die ups-lapels druckt nicht ric...
8499    an mehreren pc`s lassen sich verschiedene prgr...
Name: Translated_ShortDesc, Length: 400, dtype: object

In [19]:
df_data = Translate_text_To_en(df_data, 'Short description', 'Translated_ShortDesc' )

the JSON object must be str, bytes or bytearray, not NoneType
index is 1710 text is  
the JSON object must be str, bytes or bytearray, not NoneType
index is 6098 text is  
the JSON object must be str, bytes or bytearray, not NoneType
index is 7316 text is  
the JSON object must be str, bytes or bytearray, not NoneType
index is 7941 text is  
                                      Short description  ...                               Translated_ShortDesc
0                                           login issue  ...                                        login issue
1                                               outlook  ...                                            outlook
2                                    cant log in to vpn  ...                                 cant log in to vpn
3                         unable to access hr_tool page  ...                      unable to access hr_tool page
4                                          skype error   ...                                    

In [20]:
df_data.to_csv("/content/drive/MyDrive/AIML/projects/Capstone-NLP-Ticketing/input_data_shordesc_trans6.csv")

In [21]:
df_data['Translated_Description'] = df_data['Description']

In [22]:
df_data = Translate_text_To_en(df_data, 'Description', 'Translated_Description' )

list index out of range
index is 2915 text is  companycenter.company.com
                                      Short description  ...                             Translated_Description
0                                           login issue  ...  -verified user details.(employee# & manager na...
1                                               outlook  ...      received from: hmjdrvpb.komuaywn@gmail.com...
2                                    cant log in to vpn  ...      received from: eylqgodm.ybqkwiam@gmail.com...
3                         unable to access hr_tool page  ...                      unable to access hr_tool page
4                                          skype error   ...                                        skype error
...                                                 ...  ...                                                ...
8495                  emails not coming in from zz mail  ...      received from: avglmrts.vhqmtiua@gmail.com...
8496                           

In [23]:
df_data[df_data['Lang'] == 'de']

,Short description,Description,Caller,Assignment group,orig_desc,orig_short_desc,Lang,Translated_ShortDesc,Translated_Description
133,install driver in printer hr14 in HostName_769,install driver in printer hr14 in HostName_769,bihypqsn kbaegpcd,GRP_0,install driver in printer hr14 in HostName_769,install driver in printer hr14 in HostName_769,de,install driver in printer hr14 in HostName_769,install driver in printer hr14 in HostName_769
223,probleme mit bluescreen .,"hallo , es ist erneut passiert. der pc hat si...",vrfpyjwi nzhvgqiw,GRP_24,"hallo ,\n\nes ist erneut passiert. der pc hat ...",probleme mit bluescreen .,de,Problems with BlueScreen.,"Hello, it happened again.The PC has hung up fo..."
251,reset the password for fygrwuna gomcekzi on e-...,bitte passwort fr fygrwuna gomcekzi e-mail zur...,fygrwuna gomcekzi,GRP_0,bitte passwort fÃ¼r fygrwuna gomcekzi e-mail z...,reset the password for fygrwuna gomcekzi on e-...,de,reset the password for fygrwuna gomcekzi on e-...,Please Password for Fygrwuna Gomcekzi E-mail P...
265,netweaver funktioniert nicht mehr,received from: fcyuqvoj.ajqeidlm@gmail.com...,fcyuqvoj ajqeidlm,GRP_0,_x000D_\n_x000D_\nreceived from: fcyuqvoj.ajqe...,netweaver funktioniert nicht mehr,de,NetWeaver does not work anymore,Received from: fcyuqvoj.ajqeidlm@gmail.com Hel...
270,neues passwort fr accountname tgryhu hgygrtui,neues passwort fr accountname tgryhu hgygrtui....,nemzycxb xpsgkahw,GRP_0,neues passwort fÃ¼r accountname tgryhu hgygrtu...,neues passwort fÃ¼r accountname tgryhu hgygrtui,de,New password for AccountName TGryhu Hgygrtui,New password for AccountName TGryhu Hgygrtui.P...
...,...,...,...,...,...,...,...,...,...
8414,langsamer rechner \berprfung \niptbwdq csenjruz,langsamer rechner \berprfung \niptbwdq csenjruz,niptbwdq csenjruz,GRP_24,langsamer rechner \Ã¼berprÃ¼fung \niptbwdq cse...,langsamer rechner \Ã¼berprÃ¼fung \niptbwdq cse...,de,Slow Calculator \ Check \ Niptbwdq Csenjruz,Slow Calculator \ Check \ Niptbwdq Csenjruz
8420,mobiltelefon defekt,mobiltelefon gigaset m2 professional mit der d...,ptvdxwla tlevwmzo,GRP_33,mobiltelefon gigaset m2 professional mit der d...,mobiltelefon defekt,de,Mobile phone defect,Mobile phone Gigaset M2 Professional with the ...
8425,ich kann mein erp passwort nicht zurck setzten.,ich wei mein erp passwort nicht mehr und habe ...,wfbkucds qaxhbois,GRP_0,ich weiÃŸ mein erp passwort nicht mehr und hab...,ich kann mein erp passwort nicht zurÃ¼ck setzten.,de,I can not reset my ERP password.,I no longer know my ERP password and have 3 fa...
8439,der drucker fr die ups-lapels druckt nicht ric...,der drucker steht am platz von wckrxovs aunsgzmd,kiqrvwat gwkpxzyt,GRP_33,der drucker steht am platz von wckrxovs aunsgzmd,der drucker fÃ¼r die ups-lapels druckt nicht r...,de,The printer for the UPS lapels does not print ...,The printer is located on the place of WCKRXOV...


In [24]:
df_data.to_csv("/content/drive/MyDrive/AIML/projects/Capstone-NLP-Ticketing/input_data_trans.csv",index=False)

In [25]:
print("Translation completed")

Translation completed
